# KSegment data object tutorial

k-segment 란 k 길이의 가능한 모든 경로를 지칭한다. 이 튜토리얼에서는 이미 작성되어 있는 k-segment 파일 (`test_kseg.hdf5`)를 통해 실제로 어떤 방식으로  
사용하는지 알아본다.

In [1]:
import taxidata as td

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


일반적으로 실제 도로네트워크 상의 가능한 모든 경로의 수는 매우 많기 때문에 이를 전부 메모리에 할당하는 것이 어렵거나 불가능할 수 있다.  
KSegment는 이 문제를 해결하기 위해 고안된 클래스이다.

In [2]:
kseg = td.KSegment('test_kseg.hdf5')

k-segment는 다음과 같이 튜플 형태의 id를 갖는다.   
>`(start_node, index)` 
- start_node는 특정 segment가 시작하는 노드의 id이다.
 - index 는 start_node에서 시작하는 segment중 몇 번째에 위치하는지를 의미한다.  


전체 start_node 들을 불러오기 위해서는 다음 함수를 이용한다.

In [3]:
full_node_list = kseg.get_nodes()

In [4]:
full_node_list[:10]

[1, 10, 100, 100001, 100003, 100007, 10001, 100011, 100015, 100018]

`kseg` 객체 내에 어떤 start node 들이 포함되어 있는지 검색할 수 있다.

In [5]:
1 in kseg.get_nodes()

True

In [6]:
5 in kseg.get_nodes()

False

k-segment는 정의에 따라 엣지들의 sequence로 구현되어있다.
> - 'start' : edge의 시작 노드
- 'end' : edge의 끝 노드
- 'indices' : edge의 인덱스 (`MultiDiGraph`는 두 노드 사이에 여러 edge를 가질 수 있다.)
- 'length' : (optional) edge의 실제 지리적 길이

In [7]:
kseg[1,1]

segment([(1, 2, 0), (2, 6, 0), (6, 7, 0), (7, 6, 0), (6, 7, 0)],
        dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1')])

k-segment의 노드를 불러오려면 `node` 프로퍼티나 `nodes()` 함수를 이용한다.

In [8]:
kseg[1,1].node

array([1, 2, 6, 7, 6, 7])

## ksegment 불러오기

segment를 불러오는 방법에는 두 가지가 있다.

### 1. start_node를 이용하여 불러오기

start_node를 이용하여 불러오는 경우 `list`의 형태로 반환한다.

In [9]:
kseg[1]

[segment([(1, 2, 0), (2, 6, 0), (6, 9, 0)],
         dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1')]),
 segment([(1, 2, 0), (2, 6, 0), (6, 7, 0), (7, 6, 0), (6, 7, 0)],
         dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1')]),
 segment([(1, 2, 0), (2, 6, 0), (6, 7, 0), (7, 6, 0), (6, 9, 0)],
         dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1')]),
 segment([(1, 2, 0), (2, 6, 0), (6, 7, 0), (7, 6, 0), (6, 2, 0), (2, 3, 0)],
         dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1')]),
 segment([(1, 2, 0), (2, 6, 0), (6, 7, 0), (7, 6, 0), (6, 2, 0), (2, 6, 0)],
         dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1')]),
 segment([(1, 2, 0), (2, 6, 0), (6, 7, 0), (7, 6, 0), (6, 2, 0), (2, 1, 0),
          (1, 2, 0)],
         dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1')]),
 segment([(1, 2, 0), (2, 6, 0), (6, 2, 0), (2, 3, 0)],
         dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1')]),
 segment([(1, 2, 0),

In [10]:
import tqdm

노드의 list를 가지고 있는 경우 다음과 같이 세그먼트들을 하나의 리스트로 뽑아낼수 있다.

In [11]:
seg = []
for i in tqdm.tqdm(full_node_list[:20]):
    seg+= kseg[i]

100%|██████████| 20/20 [00:00<00:00, 238.74it/s]


k segment들의 id를 추출하는 함수는 다음과 같다.

In [12]:
import numpy as np
np.array(td.get_seg_id(*seg),dtype =np.int32) #*seg 는 seg의 컴포넌트를 각각 인수(`get_seg_id(seg[0],seg[1],seg[2],...)`)로 집어넣는 것을 의미한다.

array([[     1,      0],
       [     1,      1],
       [     1,      2],
       ...,
       [100046,   2340],
       [100046,   2341],
       [100046,   2342]])

$d_{curve}$ 계산을 위한 함수는 다음과 같다.

In [13]:
td.kseg_flattening(*seg) # return is [flatten_segment, length_array, total_number]

(array([     1,      2,      6, ..., 100514, 100431, 100929]),
 array([4, 6, 6, ..., 6, 6, 6]),
 23314)

### 2. seg_id로 불러오기

2차원 array처럼 호출 가능하다. 다만, 복수개의 segment는 가져올 수 없다.

In [14]:
kseg[1, -1]

segment([(1, 2, 0), (2, 1, 0), (1, 2, 0)],
        dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1')])

In [15]:
kseg[(1,-1)]

segment([(1, 2, 0), (2, 1, 0), (1, 2, 0)],
        dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1')])

## 미리 ksegment 로드하기 및 length 가져오기 

이 클래스는 기본적으로 스토리지에서 실시간으로 데이터를 읽어들이며 작업하므로,
모든 작업결과는 `KSegment._object`라는 `dict`에 저장된다.  
따라서 메모리가 부족해지는 순간이 올 수 있기때문에 여기서는 메모리를 수동적으로 관리하는 방법과 추가적으로  
length 어트리뷰트를 함께 로드하는 방법을 소개한다.

### 메모리 해제하기

위의 예제에서 이미 많은 start_node들을 불러서 작업했기 때문에 이미 _object에 로드한 segment들이 저장되어있다.

In [16]:
type(kseg._object)

dict

In [17]:
kseg._object[1]

segment([[( 1,  2, 0), ( 2,  6, 0), ( 6,  9, 0), ( 9, -1, 0),
          (-1, -1, 0), (-1, -1, 0), (-1, -1, 0)],
         [( 1,  2, 0), ( 2,  6, 0), ( 6,  7, 0), ( 7,  6, 0),
          ( 6,  7, 0), ( 7, -1, 0), (-1, -1, 0)],
         [( 1,  2, 0), ( 2,  6, 0), ( 6,  7, 0), ( 7,  6, 0),
          ( 6,  9, 0), ( 9, -1, 0), (-1, -1, 0)],
         [( 1,  2, 0), ( 2,  6, 0), ( 6,  7, 0), ( 7,  6, 0),
          ( 6,  2, 0), ( 2,  3, 0), ( 3, -1, 0)],
         [( 1,  2, 0), ( 2,  6, 0), ( 6,  7, 0), ( 7,  6, 0),
          ( 6,  2, 0), ( 2,  6, 0), ( 6, -1, 0)],
         [( 1,  2, 0), ( 2,  6, 0), ( 6,  7, 0), ( 7,  6, 0),
          ( 6,  2, 0), ( 2,  1, 0), ( 1,  2, 0)],
         [( 1,  2, 0), ( 2,  6, 0), ( 6,  2, 0), ( 2,  3, 0),
          ( 3, -1, 0), (-1, -1, 0), (-1, -1, 0)],
         [( 1,  2, 0), ( 2,  6, 0), ( 6,  2, 0), ( 2,  6, 0),
          ( 6, -1, 0), (-1, -1, 0), (-1, -1, 0)],
         [( 1,  2, 0), ( 2,  6, 0), ( 6,  2, 0), ( 2,  1, 0),
          ( 1,  2, 0), ( 2, -1, 0), (-1, -

`clear()` 함수를 이용하여 `_object`를 해제할 수 있다.

In [18]:
kseg.clear()

In [19]:
kseg._object

{}

### load 함수를 이용하여 미리 k segment불러오기

미리 사용하고자하는 node들을 리스트나 array를 이용하여 불러들일 수 있다. 

다만, 모든 노드가 꼭 ksegment를 가지고 있는 것은 아니기 때문에 ksegment에 존재하는 노드만 간추릴 필요가 있다.

In [20]:
zero_to_nine = np.arange(9)

`mask`함수는 이런 일을 위해 필요한 함수이다.

In [21]:
kseg.mask(zero_to_nine) #백만개 검사하는데 0.1초 안걸림

array([False,  True,  True,  True, False, False,  True,  True, False])

In [22]:
zero_to_nine[kseg.mask(zero_to_nine)]

array([1, 2, 3, 6, 7])

혹은 '여과' 함수 `filtrate`를 이용하여 위의 작업을 한번에 처리할 수 있다.

In [23]:
kseg.filtrate(zero_to_nine)

array([1, 2, 3, 6, 7])

주어진 노드를 메모리에 추가하는 것은 `loads(nodes)` 함수를 이용해서 할 수 있다. 이 경우 자동적으로 길이 데이터를 추가로 가져온다.

In [25]:
kseg.loads(kseg.filtrate(full_node_list[:50]))

100%|██████████| 50/50 [00:00<00:00, 687.03it/s]


In [26]:
kseg[1]

[segment([(1, 2, 0,   82.62997), (2, 6, 0,   77.25552),
          (6, 9, 0, 3904.015  )],
         dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1'), ('length', '<f4')]),
 segment([(1, 2, 0, 82.62997 ), (2, 6, 0, 77.25552 ), (6, 7, 0, 17.483225),
          (7, 6, 0, 17.483225), (6, 7, 0, 17.483225)],
         dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1'), ('length', '<f4')]),
 segment([(1, 2, 0,   82.62997 ), (2, 6, 0,   77.25552 ),
          (6, 7, 0,   17.483225), (7, 6, 0,   17.483225),
          (6, 9, 0, 3904.015   )],
         dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1'), ('length', '<f4')]),
 segment([(1, 2, 0,  82.62997 ), (2, 6, 0,  77.25552 ),
          (6, 7, 0,  17.483225), (7, 6, 0,  17.483225),
          (6, 2, 0,  77.25552 ), (2, 3, 0, 121.804085)],
         dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1'), ('length', '<f4')]),
 segment([(1, 2, 0, 82.62997 ), (2, 6, 0, 77.25552 ), (6, 7, 0, 17.483225),
          (7, 6, 0, 17.

만약, 길이가 필요하다면 `loads`에 length 인자를 변경하면 같이 로드할지 결정할 수 있다.

In [27]:
kseg.loads(full_node_list[:50],length=False)

100%|██████████| 50/50 [00:00<00:00, 928.41it/s]


In [28]:
kseg[1]

[segment([(1, 2, 0), (2, 6, 0), (6, 9, 0)],
         dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1')]),
 segment([(1, 2, 0), (2, 6, 0), (6, 7, 0), (7, 6, 0), (6, 7, 0)],
         dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1')]),
 segment([(1, 2, 0), (2, 6, 0), (6, 7, 0), (7, 6, 0), (6, 9, 0)],
         dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1')]),
 segment([(1, 2, 0), (2, 6, 0), (6, 7, 0), (7, 6, 0), (6, 2, 0), (2, 3, 0)],
         dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1')]),
 segment([(1, 2, 0), (2, 6, 0), (6, 7, 0), (7, 6, 0), (6, 2, 0), (2, 6, 0)],
         dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1')]),
 segment([(1, 2, 0), (2, 6, 0), (6, 7, 0), (7, 6, 0), (6, 2, 0), (2, 1, 0),
          (1, 2, 0)],
         dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1')]),
 segment([(1, 2, 0), (2, 6, 0), (6, 2, 0), (2, 3, 0)],
         dtype=[('start', '<i4'), ('end', '<i4'), ('indices', 'i1')]),
 segment([(1, 2, 0),